In [1]:
import pandas as pd
import numpy as np
from ebmdatalab import bq

In [3]:
sql = '''SELECT
  extract(year from month) AS year,
  bnf_code AS bnfcode,
  bnf_name AS bnfname,
  ccg.org_type AS org_type,
  SUM(items) AS total_items,
  SUM(quantity) AS total_qty,
  SUM(actual_cost) AS actual_cost
FROM
  ebmdatalab.hscic.normalised_prescribing_standard AS presc
JOIN
  hscic.ccgs AS ccg #this joins to our CCG organisational data
ON
  presc.pct = ccg.code
WHERE
  bnf_code LIKE "0410020C0%"  
GROUP BY
  year,
  bnf_code,
  bnf_name,
  org_type'''
    
df_varenicline = bq.cached_read(sql, csv_path='df_varenicline.csv', use_cache=True)  # add `use_cache=False` to override
df_varenicline.head()

C:\Users\bmackenna\Anaconda3\lib\site-packages\pandas\io\gbq.py:149: FutureWarning: verbose is deprecated and will be removed in a future version. Set logging level in order to vary verbosity
  credentials=credentials, verbose=verbose, private_key=private_key)


,year,bnfcode,bnfname,org_type,total_items,total_qty,actual_cost
0,2014,0410020C0AAABAB,Varenicline Tart_Tab 1mg,Unknown,2513,83362,75126.43000
1,2019,0410020C0BBACAC,Champix_2Wk Tt Init Pack (Tab 0.5mg/1mg),CCG,2076,2099,53233.43212
2,2018,0410020C0AAACAC,Varenicline Tart_2Wk Init(Tab 0.5mg/1mg),Unknown,2059,2082,52786.42973
3,2019,0410020C0AAAAAA,Varenicline Tart_Tab 0.5mg,Unknown,22,673,611.04191
4,2012,0410020C0BBACAC,Champix_Titration Pack (Tab 0.5mg/1mg),PCT,43,43,1084.94000


In [4]:
## ensuring the format is consistent for pounds and pence
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [5]:
### grouping the total cost by year
totalcost = df_varenicline.groupby(['year'])['actual_cost'].sum()
totalcost

year
2011    4369229.34
2012   27604089.08
2013   22982963.13
2014   18399874.32
2015   16253500.84
2016   13383274.38
2017   12805621.15
2018   10672671.14
2019     874189.82
Name: actual_cost, dtype: float64

In [6]:
### grouping the total items by year
totalitems = df_varenicline.groupby(['year'])['total_items'].sum()
totalitems

year
2011    138219
2012    885330
2013    734317
2014    585281
2015    514644
2016    417345
2017    395247
2018    325229
2019     26772
Name: total_items, dtype: int64

In [7]:
## Selecting only nonCCG organisations to work with
nonccg_varenicline  = df_varenicline.loc[df_varenicline.org_type !='CCG']
nonccg_varenicline.head()

,year,bnfcode,bnfname,org_type,total_items,total_qty,actual_cost
0,2014,0410020C0AAABAB,Varenicline Tart_Tab 1mg,Unknown,2513,83362,75126.43
2,2018,0410020C0AAACAC,Varenicline Tart_2Wk Init(Tab 0.5mg/1mg),Unknown,2059,2082,52786.43
3,2019,0410020C0AAAAAA,Varenicline Tart_Tab 0.5mg,Unknown,22,673,611.04
4,2012,0410020C0BBACAC,Champix_Titration Pack (Tab 0.5mg/1mg),PCT,43,43,1084.94
5,2016,0410020C0BBACAC,Champix_Titration Pack (Tab 0.5mg/1mg),Unknown,10,10,252.39


In [12]:
## Restricting data to 2018 calendar year
nonccg_varenicline_2018 = nonccg_varenicline.loc[(nonccg_varenicline["year"] == 2018)]
nonccg_varenicline_2018.head()

,year,bnfcode,bnfname,org_type,total_items,total_qty,actual_cost
2,2018,0410020C0AAACAC,Varenicline Tart_2Wk Init(Tab 0.5mg/1mg),Unknown,2059,2082,52786.43
27,2018,0410020C0AAABAB,Varenicline Tart_Tab 1mg,Unknown,3559,109938,99511.09
49,2018,0410020C0AAAAAA,Varenicline Tart_Tab 0.5mg,Unknown,278,7306,6636.92
64,2018,0410020C0BBABAB,Champix_Tab 1mg,Unknown,577,16843,15244.69
67,2018,0410020C0BBACAC,Champix_2Wk Tt Init Pack (Tab 0.5mg/1mg),Unknown,325,327,8290.07


In [13]:
nonccg_varenicline_2018.sum()

year                                                       14126
bnfcode        0410020C0AAACAC0410020C0AAABAB0410020C0AAAAAA0...
bnfname        Varenicline Tart_2Wk Init(Tab 0.5mg/1mg)Vareni...
org_type       UnknownUnknownUnknownUnknownUnknownUnknownUnknown
total_items                                                 6855
total_qty                                                 138405
actual_cost                                            184238.79
dtype: object